In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
sys.path.append('..')
from scipy.constants import hbar, e, h

# Single transmon qubit

Calculations for weakly anharmonic systems can be divide into two main parts:
1. Calculations for linear circuit, where all nonlinaer elements are replaced by linear inductances

2. Calculation of Kerr matrix, which defines anharmonicities and dispersive shifts

## Create linear circuit

In [2]:
from tlsim2.lumped import LumpedTwoTerminal
from tlsim2.lumped import JosephsonJunction

Cq = 73e-15
EJ = 16e9 * h

# qubit capacitor
capacitor_qubit = LumpedTwoTerminal(name='Cq', c=Cq)

# qubit nonlinear element
jj =  JosephsonJunction(name='JJ', ej=EJ)

from tlsim2.circuit import Circuit

linear_circuit = Circuit()
linear_circuit.add_element(capacitor_qubit, {'i': 0, 'o': 1})
linear_circuit.add_element(jj, {'i': 0, 'o': 1})
linear_circuit.short(0)

w, v, node_names = linear_circuit.compute_system_modes()
decays = np.real(w/1e9)*2
frequencies = np.imag(w/(2*np.pi))/1e9
decays = np.real(w/1e9)*2
decays = decays[frequencies > 0]
frequencies = frequencies[frequencies > 0]
frequencies, decays 

(array([5.8278845]), array([0.]))

## Create nonlinear circuit

In [3]:
print(linear_circuit.nonlinear_elements)

from tlsim2.quantum_circuit import QuantumCircuit
nonlinear_circuit = QuantumCircuit(linear_circuit)

print(f"Number of modes in the system is {nonlinear_circuit.num_modes} and \
number of nonlinaer elements is {nonlinear_circuit.num_nonlinear_elements}")

OrderedDict([('JJ', <tlsim2.lumped.JosephsonJunction object at 0x0000029BB5277340>)])
Number of modes in the system is 1 and number of nonlinaer elements is 1


In [4]:
nonlinear_circuit.get_epr_coefficients(), np.sum(nonlinear_circuit.get_epr_coefficients())

(array([[1.]]), 0.9999999999999999)

In [5]:
kerr_matrix, anharmonicities = nonlinear_circuit.get_kerr_matrix()
kerr_matrix / (2 * np.pi), anharmonicities / (2 * np.pi)

f_p = frequencies[0]
alpha = - anharmonicities[0] / (2 * np.pi) / 1e6

print(f"Plasma frequency for transmon qubit is {f_p :.2f} GHz and \
anharmonicity {alpha :.2f} MHz, therefore qubit transition is {f_p + alpha / 1e3 :.2f} GHz")

Plasma frequency for transmon qubit is 5.83 GHz and anharmonicity -265.35 MHz, therefore qubit transition is 5.56 GHz


## Compare result with theory

Plasma frequency of transmon qubit is defined as $\omega_{p} =  \sqrt{8 E_C E_J}$,

anharmonicity for transmon qubit is $\alpha = -E_C$

and transition $\omega_{01} =  \omega_p + \alpha$ 

In [6]:
EC = e ** 2 / (Cq) / 2 / h
f_p = np.sqrt(8 * EC * EJ / h)

print(f"Plasma frequency for transmon qubit is {f_p / 1e9 :.2f} GHz and anharmonicity {- EC / 1e6 :.2f} MHz")

Plasma frequency for transmon qubit is 5.83 GHz and anharmonicity -265.35 MHz


# Circuit of coupled transmon qubit and $\lambda / 4$ resonator

## In linear approximation

In [7]:
from tlsim2.tl import MultiTransmissionLine
from tlsim2.tl import default_tl_basis

from tlsim2.lumped import LumpedTwoTerminal
from tlsim2.lumped import JosephsonJunction

ll = [[416.120e-9]]  # per-unit-length inductance for a 50-ohm TL on cold silicon
cl = [[166.448e-12]]  # per-unit-length capacitance for a 50-ohm TL on cold silicon

vp = 1/np.sqrt(np.asarray(cl)*np.asarray(ll))
length = 4e-3

CJ = 73e-15
EJ = 16e9 * h
Cc = 4e-15

# resonator transmission line
default_basis1 = default_tl_basis(1, n_harmonics=1)
tl = MultiTransmissionLine('TL', n=1, l=length, ll=ll,
                           cl=cl, basis=default_basis1)

# qubit capacitor
capacitor_qubit = LumpedTwoTerminal(name='CJ', c=CJ)

# qubit nonlinear element
jj = JosephsonJunction(name='JJ', ej=EJ)

# coupling capacitor
capacitor_coupl = LumpedTwoTerminal(name='Cc', c=Cc)

In [8]:
from tlsim2.circuit import Circuit

linear_circuit = Circuit()
linear_circuit.add_element(capacitor_qubit, {'i': 0, 'o': 1})
linear_circuit.add_element(jj, {'i': 0, 'o': 1})
linear_circuit.add_element(tl, {'i0': 2, 'o0': 0})
linear_circuit.add_element(capacitor_coupl, {'i': 1, 'o': 2})
linear_circuit.short(0)

w, v, node_names = linear_circuit.compute_system_modes()
decays = np.real(w/1e9)*2
frequencies = np.imag(w/(2*np.pi))/1e9
decays = np.real(w/1e9)*2
decays = decays[frequencies > 0]
frequencies = frequencies[frequencies > 0]
frequencies, decays 

(array([47.91402092, 22.88593126,  5.67185095,  7.47048363]),
 array([0., 0., 0., 0.]))

## Nonlinear circuit

Check all nonlinear elements in the circuit

In [9]:
linear_circuit.nonlinear_elements

OrderedDict([('JJ', <tlsim2.lumped.JosephsonJunction at 0x29bd57050d0>)])

Now one can create quantum nonlinear circuit from this linear circuit

In [10]:
from tlsim2.quantum_circuit import QuantumCircuit
nonlinear_circuit = QuantumCircuit(linear_circuit)

print(f"Number of modes in the system is {nonlinear_circuit.num_modes} and \
number of nonlinear elements is {nonlinear_circuit.num_nonlinear_elements}")

Number of modes in the system is 4 and number of nonlinear elements is 1


Calculate EPR matrix $\bf{p}$ which is defined as $p_{mj} =  \frac{\text{Inductive energy in junction }j}{\text{Half of total energy if the mode } m}$ (https://www.nature.com/articles/s41534-021-00461-8)

In [11]:
 nonlinear_circuit.get_epr_coefficients(), np.sum(nonlinear_circuit.get_epr_coefficients())

(array([[4.25440084e-05],
        [4.53254788e-05],
        [9.97914284e-01],
        [1.99784680e-03]]),
 0.9999999999999999)

Kerr matrix defines the Hamiltonian function in the following form
$$H = \sum_{m} \hbar \omega_m a_m ^ {\dagger} a_m + \frac{1}{2} \sum_{n, m} \chi_{m n} a_m ^ {\dagger} a_m a_n ^ {\dagger} a_n$$
From this approach anharmonicity of mode $m$ is calculated as $A_m =  \frac{1}{2} \chi_{mm}$

In [12]:
kerr_matrix, anharmonicities = nonlinear_circuit.get_kerr_matrix()
kerr_matrix / (2 * np.pi), anharmonicities / (2 * np.pi)

(array([[6.49265121e+01, 3.30393914e+01, 1.80276492e+05, 4.75370197e+02],
        [3.30393914e+01, 1.68128758e+01, 9.17379570e+04, 2.41903369e+02],
        [1.80276492e+05, 9.17379570e+04, 5.00559979e+08, 1.31992415e+06],
        [4.75370197e+02, 2.41903369e+02, 1.31992415e+06, 3.48050152e+03]]),
 array([3.24632561e+01, 8.40643791e+00, 2.50279989e+08, 1.74025076e+03]))